
# Prerequsites

In [0]:
storage_account = "azurestgmaveric"
container_name = "users"
storage_account_conf_key = (
    f"fs.azure.account.key.{storage_account}.dfs.core.windows.net"
)
storage_account_conf_key_value = "vR3zRwrjFujNOj8lvCqL5E5VV8ifNxpAR8BMvCGuvLjawr10b6zzi3uBq7wNiPvizR1KjlhmaZqW+ASt84bxTA=="
spark.conf.set(storage_account_conf_key, storage_account_conf_key_value)

## Load data into DF

In [0]:

file_names = "/raw/{users_001.csv}"
users_dataset_path = (
    f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/{file_names}"
)
user_df = spark.read.csv(path=users_dataset_path, header=True, inferSchema=True)
print(f"Total no of records:{user_df.count()}")

Total no of records:500


## Create schema

In [0]:
%sql
-- DROP CATALOG IF EXISTS quickstart_catalog CASCADE;
-- CREATE CATALOG IF NOT EXISTS quickstart_catalog;
DROP SCHEMA IF EXISTS quickstart_catalog.quickstart_schema CASCADE;
CREATE SCHEMA IF NOT EXISTS quickstart_catalog.quickstart_schema;
USE quickstart_catalog.quickstart_schema;
show SCHEMAS

databaseName default information_schema quickstart_schema

#  Transaction 01-  Write data to Delta Table

In [0]:
user_df.write.format("delta").mode("append").saveAsTable("user_info")

## Read Version Table

In [0]:
%sql
DESCRIBE HISTORY user_info;

version timestamp userId userName operation operationParameters job notebook clusterId readVersion isolationLevel isBlindAppend operationMetrics userMetadata engineInfo 0 2024-07-05T11:23:07Z 2005024220910796 naveen_1719215901936@npmavericsystems.onmicrosoft.com CREATE TABLE AS SELECT Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {"delta.enableDeletionVectors":"true"}, statsOnLoad -> false) null List(984221041466094) 0705-091044-8zcq4d8j null WriteSerializable true Map(numFiles -> 1, numOutputRows -> 500, numOutputBytes -> 28435) null Databricks-Runtime/15.3.x-cpu-ml-scala2.12

In [0]:
from pyspark.sql.functions import *
spark.read.table("user_info").filter(col("city") == "Bangalore").display()

id name dob email gender country region city asset marital_status 263 Nelle Blake 2024-09-17 nelleblake5744@ymail.com Transgender India Karnataka Bangalore 908573 Married

# Transaction 02 : Update

In [0]:
%sql

UPDATE user_info SET city ="Bengaluru"
where city="Bangalore"

num_affected_rows 1

In [0]:
%sql
DESCRIBE HISTORY user_info;

version timestamp userId userName operation operationParameters job notebook clusterId readVersion isolationLevel isBlindAppend operationMetrics userMetadata engineInfo 1 2024-07-05T11:24:11Z 2005024220910796 naveen_1719215901936@npmavericsystems.onmicrosoft.com UPDATE Map(predicate -> ["(city#12051 = Bangalore)"]) null List(984221041466094) 0705-091044-8zcq4d8j 0 WriteSerializable false Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2070, numDeletionVectorsUpdated -> 0, scanTimeMs -> 794, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 2978, rewriteTimeMs -> 1276) null Databricks-Runtime/15.3.x-cpu-ml-scala2.12 0 2024-07-05T11:23:07Z 2005024220910796 naveen_1719215901936@npmavericsystems.onmicrosoft.com CREATE TABLE AS SELECT Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {"delta.enableDeletionVectors":"true"}, statsOnLoad -> false) null List(984221041466094) 0705-091044-8zcq4d8j null WriteSerializable true Map(numFiles -> 1, numOutputRows -> 500, numOutputBytes -> 28435) null Databricks-Runtime/15.3.x-cpu-ml-scala2.12

# Using Versioning

In [0]:
from pyspark.sql.functions import *
spark.read.table("user_info").filter(col("city") == "Bengaluru").display()

id name dob email gender country region city asset marital_status 263 Nelle Blake 2024-09-17 nelleblake5744@ymail.com Transgender India Karnataka Bengaluru 908573 Married

In [0]:
spark.read.option("timeStampAsOf","2024-09").table("user_info").filter(col("city") == "Bangalore").display()

id name dob email gender country region city asset marital_status 263 Nelle Blake 2024-09-17 nelleblake5744@ymail.com Transgender India Karnataka Bangalore 908573 Married

In [0]:
%sql

SELECT * FROM user_info VERSION AS OF 0 
where city='Bangalore';

id name dob email gender country region city asset marital_status 263 Nelle Blake 2024-09-17 nelleblake5744@ymail.com Transgender India Karnataka Bangalore 908573 Married

In [0]:
%sql
DESCRIBE FORMATTED user_info;

col_name data_type comment id int null name string null dob date null email string null gender string null country string null region string null city string null asset int null marital_status string null # Delta Statistics Columns Column Names city, name, marital_status, email, dob, country, id, region, asset, gender Column Selection Method first-32 # Detailed Table Information Catalog quickstart_catalog Database quickstart_schema Table user_info Created Time Fri Jul 05 11:23:09 UTC 2024 Last Access UNKNOWN Created By Spark Type MANAGED Location abfss://unity-catalog-storage@dbstoragebaysmlm4fi54k.dfs.core.windows.net/3381240464789137/__unitystorage/catalogs/75fdf4fc-0ac1-4ae7-bda5-690e9166e26c/tables/32984a56-3705-4cd9-88fb-54a8bc93f529 Provider delta Owner naveen_1719215901936@npmavericsystems.onmicrosoft.com Is_managed_location true Table Properties [delta.enableDeletionVectors=true,delta.feature.deletionVectors=supported,delta.minReaderVersion=3,delta.minWriterVersion=7]

In [0]:
%sql


SELECT * FROM user_info FROM 'abfss://unity-catalog-storage@dbstoragebaysmlm4fi54k.dfs.core.windows.net/3381240464789137/__unitystorage/catalogs/75fdf4fc-0ac1-4ae7-bda5-690e9166e26c/tables/32984a56-3705-4cd9-88fb-54a8bc93f529/@v1' 
where city='Bangalore';

In [0]:
%sql

SELECT * FROM user_info TIMESTAMP AS OF ''
where city='Bangalore';

In [0]:
%sql
DESCRIBE HISTORY user_info;

version timestamp userId userName operation operationParameters job notebook clusterId readVersion isolationLevel isBlindAppend operationMetrics userMetadata engineInfo 1 2024-07-05T11:24:11Z 2005024220910796 naveen_1719215901936@npmavericsystems.onmicrosoft.com UPDATE Map(predicate -> ["(city#12051 = Bangalore)"]) null List(984221041466094) 0705-091044-8zcq4d8j 0 WriteSerializable false Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2070, numDeletionVectorsUpdated -> 0, scanTimeMs -> 794, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 2978, rewriteTimeMs -> 1276) null Databricks-Runtime/15.3.x-cpu-ml-scala2.12 0 2024-07-05T11:23:07Z 2005024220910796 naveen_1719215901936@npmavericsystems.onmicrosoft.com CREATE TABLE AS SELECT Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {"delta.enableDeletionVectors":"true"}, statsOnLoad -> false) null List(984221041466094) 0705-091044-8zcq4d8j null WriteSerializable true Map(numFiles -> 1, numOutputRows -> 500, numOutputBytes -> 28435) null Databricks-Runtime/15.3.x-cpu-ml-scala2.12

In [0]:
%sql

SELECT * FROM user_info where city = "Bengaluru";

id name dob email gender country region city asset marital_status 263 Nelle Blake 2024-09-17 nelleblake5744@ymail.com Transgender India Karnataka Bengaluru 908573 Married

In [0]:
%sql

RESTORE TABLE user_info TO VERSION AS OF 0;

table_size_after_restore num_of_files_after_restore num_removed_files num_restored_files removed_files_size restored_files_size 28435 1 2 1 31413 28435

In [0]:
%sql

SELECT * FROM user_info where city = "Bengaluru";

id name dob email gender country region city asset marital_status

In [0]:
%sql
DESCRIBE HISTORY user_info;

version timestamp userId userName operation operationParameters job notebook clusterId readVersion isolationLevel isBlindAppend operationMetrics userMetadata engineInfo 2 2024-07-05T11:52:30Z 2005024220910796 naveen_1719215901936@npmavericsystems.onmicrosoft.com RESTORE Map(version -> 0, timestamp -> null) null List(984221041466094) 0705-091044-8zcq4d8j 1 Serializable false Map(numRestoredFiles -> 1, removedFilesSize -> 31413, numRemovedFiles -> 2, restoredFilesSize -> 28435, numOfFilesAfterRestore -> 1, tableSizeAfterRestore -> 28435) null Databricks-Runtime/15.3.x-cpu-ml-scala2.12 1 2024-07-05T11:24:11Z 2005024220910796 naveen_1719215901936@npmavericsystems.onmicrosoft.com UPDATE Map(predicate -> ["(city#12051 = Bangalore)"]) null List(984221041466094) 0705-091044-8zcq4d8j 0 WriteSerializable false Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2070, numDeletionVectorsUpdated -> 0, scanTimeMs -> 794, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 2978, rewriteTimeMs -> 1276) null Databricks-Runtime/15.3.x-cpu-ml-scala2.12 0 2024-07-05T11:23:07Z 2005024220910796 naveen_1719215901936@npmavericsystems.onmicrosoft.com CREATE TABLE AS SELECT Map(partitionBy -> [], clusterBy -> [], description -> null, isManaged -> true, properties -> {"delta.enableDeletionVectors":"true"}, statsOnLoad -> false) null List(984221041466094) 0705-091044-8zcq4d8j null WriteSerializable true Map(numFiles -> 1, numOutputRows -> 500, numOutputBytes -> 28435) null Databricks-Runtime/15.3.x-cpu-ml-scala2.12